In [1]:
import hail as hl
import numpy as np
import datetime
import os

n_chunks = 2
batch = 1

output_bucket = 'gs://nbaya/rg_sex/'


In [2]:
heritable_phens_tb = hl.import_table('gs://nbaya/split/ukb_phenos_for_sex_rg.tsv')
#phens_tb = hl.import_table('/Users/nbaya/Downloads/ukb_phenos_for_sex_rg.tsv')
heritable_phens_ls = [x.phenotype.strip('_irnt') for x in heritable_phens_tb.select('phenotype').collect()] #list of heritable phenotypes for UKBB round 2

phen_tb_all = hl.import_table('gs://phenotype_31063/ukb31063.phesant_phenotypes.both_sexes.tsv.bgz',
                                      missing='',impute=True,types={'"userId"': hl.tstr}).rename({ '"userId"': 's'})
withdrawn = hl.import_table('gs://nbaya/w31063_20181016.csv',missing='',no_header=True)
withdrawn_set = set(withdrawn.f0.take(withdrawn.count()))
phen_tb_all = phen_tb_all.filter(hl.literal(withdrawn_set).contains(phen_tb_all['s']),keep=False) 
phen_tb_all = phen_tb_all.key_by('s')


Initializing Spark and Hail with default parameters...
Running on Apache Spark version 2.2.1
SparkUI available at http://10.128.0.104:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.7-2b1cb1ffc672
LOGGING: writing to /home/hail/hail-20190115-1525-0.2.7-2b1cb1ffc672.log
2019-01-15 15:26:08 Hail: INFO: Reading table with no type imputation
  Loading column 'phenotype' as type 'str' (type not specified)
  Loading column 'description' as type 'str' (type not specified)
  Loading column 'h2_liability' as type 'str' (type not specified)
  Loading column 'h2_liability_se' as type 'str' (type not specified)
  Loading column 'h2_observed' as type 'str' (type not specified)
  Loading column 'h2_observed_se' as type 'str' (type not specified)
  Loading column 'h2_p' as type 'str' (type not specified)
  Loading column 'h2_sig' as type 'str' (type not specified)
  Loading column 'conf_simple' as type 'str' (type not specified)
  Loading

2019-01-15 15:26:51 Hail: INFO: Reading table with no type imputation
  Loading column 'f0' as type 'str' (type not specified)



In [3]:
# phen_tb_all_cols = [x.strip('"') for x in list(phen_tb_all.row)]

# phens_ls = list(set(heritable_phens_ls).intersection(set(phen_tb_all_cols)))

In [89]:
# phen = phens_ls[0]
# phen_tb_all = phen_tb_all.rename({'"'+phen+'"': 'phen'})
# phen_tb = phen_tb_all.select(phen_tb_all['phen'])

In [3]:
# variants = hl.read_matrix_table('gs://nbaya/split/ukb31063.hm3_variants.gwas_samples_repart.mt') #hm3 variants

In [5]:
icd_phens = hl.import_table('gs://nbaya/sex_linreg/icd10_phenotypes.both_sexes.tsv.bgz',
                                      missing='',impute=True,types={'s': hl.tstr})
finngen = hl.import_table('gs://ukb31063-mega-gwas/phenotype-files/curated-phenotypes/2018-04-06_ukb-finngen-pheno-for-analysis.tsv',
                                      missing='',impute=True).rename({'eid':'s'})
phesant_more = hl.import_table('gs://ukb31063-mega-gwas/phenotype-files/still-more-phesant/cts_irnt.both_sexes.tsv',
                                      missing='',impute=True)

2019-01-14 14:01:58 Hail: INFO: Reading table to impute column types
2019-01-14 14:03:05 Hail: INFO: Finished type imputation
  Loading column 's' as type 'str' (user-specified)
  Loading column 'T39' as type 'bool' (imputed)
  Loading column 'F13' as type 'bool' (imputed)
  Loading column 'F34' as type 'bool' (imputed)
  Loading column 'P05' as type 'bool' (imputed)
  Loading column 'R30' as type 'bool' (imputed)
  Loading column 'S98' as type 'bool' (imputed)
  Loading column 'D67' as type 'bool' (imputed)
  Loading column 'L42' as type 'bool' (imputed)
  Loading column 'E43' as type 'bool' (imputed)
  Loading column 'H66' as type 'bool' (imputed)
  Loading column 'I73' as type 'bool' (imputed)
  Loading column 'N34' as type 'bool' (imputed)
  Loading column 'L10' as type 'bool' (imputed)
  Loading column 'Q83' as type 'bool' (imputed)
  Loading column 'Z24' as type 'bool' (imputed)
  Loading column 'L87' as type 'bool' (imputed)
  Loading column 'Z64' as type 'bool' (imputed)
  Load

2019-01-14 14:03:05 Hail: INFO: Reading table to impute column types
2019-01-14 14:03:24 Hail: INFO: Finished type imputation
  Loading column 'eid' as type 'int32' (imputed)
  Loading column 'ICDMAIN_ANY_ENTRY' as type 'int32' (imputed)
  Loading column 'XI_DIGESTIVE' as type 'int32' (imputed)
  Loading column 'XVIII_MISCFINDINGS' as type 'int32' (imputed)
  Loading column 'XIII_MUSCULOSKELET' as type 'int32' (imputed)
  Loading column 'XIV_GENITOURINARY' as type 'int32' (imputed)
  Loading column 'II_NEOPLASM' as type 'int32' (imputed)
  Loading column 'IX_CIRCULATORY' as type 'int32' (imputed)
  Loading column 'XXI_HEALTHFACTORS' as type 'int32' (imputed)
  Loading column 'XIX_INJURY_POISON' as type 'int32' (imputed)
  Loading column 'VII_EYE_ADNEXA' as type 'int32' (imputed)
  Loading column 'M13_OTHERJOINT' as type 'int32' (imputed)
  Loading column 'XII_SKIN_SUBCUTAN' as type 'int32' (imputed)
  Loading column 'ILD_DIFF_DG' as type 'int32' (imputed)
  Loading column 'COPD_EXCL' a

2019-01-14 14:03:26 Hail: INFO: Reading table to impute column types
2019-01-14 14:03:31 Hail: INFO: Finished type imputation
  Loading column 'userId' as type 'int32' (imputed)
  Loading column '84' as type 'float64' (imputed)
  Loading column '93' as type 'float64' (imputed)
  Loading column '94' as type 'float64' (imputed)
  Loading column '102' as type 'float64' (imputed)
  Loading column '399' as type 'float64' (imputed)
  Loading column '400' as type 'float64' (imputed)
  Loading column '404' as type 'float64' (imputed)
  Loading column '3062' as type 'float64' (imputed)
  Loading column '3063' as type 'float64' (imputed)
  Loading column '3064' as type 'float64' (imputed)
  Loading column '4079' as type 'float64' (imputed)
  Loading column '4080' as type 'float64' (imputed)
  Loading column '4230' as type 'float64' (imputed)
  Loading column '4233' as type 'float64' (imputed)
  Loading column '4241' as type 'float64' (imputed)
  Loading column '4244' as type 'float64' (imputed)


In [3]:
phesant_still_more1 = hl.import_table('gs://ukb31063-mega-gwas/phenotype-files/still-more-phesant/neale_lab_parsed_and_restricted_to_QCed_samples_cat_variables_both_sexes.1.tsv',
                                      missing='',impute=True,no_header=False)

2019-01-15 13:30:45 Hail: INFO: Reading table to impute column types
2019-01-15 13:30:54 Hail: INFO: Finished type imputation
  Loading column 'all_sexes$userId' as type 'int32' (imputed)
  Loading column '134' as type 'int32' (imputed)
  Loading column '135' as type 'int32' (imputed)
  Loading column '136' as type 'int32' (imputed)
  Loading column '137' as type 'int32' (imputed)
  Loading column '398' as type 'int32' (imputed)
  Loading column '403' as type 'int32' (imputed)
  Loading column '709' as type 'int32' (imputed)
  Loading column '728' as type 'int32' (imputed)
  Loading column '738' as type 'int32' (imputed)
  Loading column '777' as type 'int32' (imputed)
  Loading column '806' as type 'int32' (imputed)
  Loading column '816' as type 'int32' (imputed)
  Loading column '826' as type 'int32' (imputed)
  Loading column '845' as type 'int32' (imputed)
  Loading column '864' as type 'int32' (imputed)
  Loading column '884' as type 'int32' (imputed)
  Loading column '894' as ty

In [4]:
phesant_still_more2 = hl.import_table('gs://ukb31063-mega-gwas/phenotype-files/still-more-phesant/neale_lab_parsed_and_restricted_to_QCed_samples_cat_variables_both_sexes.2.tsv',
                                      missing='',impute=True)

2019-01-15 13:30:55 Hail: INFO: Reading table to impute column types
2019-01-15 13:31:21 Hail: INFO: Finished type imputation
  Loading column 'all_sexes$userId' as type 'int32' (imputed)
  Loading column '5993' as type 'int32' (imputed)
  Loading column '6039' as type 'int32' (imputed)
  Loading column '20018' as type 'int32' (imputed)
  Loading column '20077' as type 'int32' (imputed)
  Loading column '20128' as type 'int32' (imputed)
  Loading column '20403' as type 'int32' (imputed)
  Loading column '20407' as type 'int32' (imputed)
  Loading column '20408' as type 'int32' (imputed)
  Loading column '20409' as type 'int32' (imputed)
  Loading column '20412' as type 'int32' (imputed)
  Loading column '20413' as type 'int32' (imputed)
  Loading column '20414' as type 'int32' (imputed)
  Loading column '20416' as type 'int32' (imputed)
  Loading column '20418' as type 'int32' (imputed)
  Loading column '20436' as type 'int32' (imputed)
  Loading column '20438' as type 'int32' (imputed

In [12]:
phesant_still_more3 = hl.import_table('gs://ukb31063-mega-gwas/phenotype-files/still-more-phesant/neale_lab_parsed_and_restricted_to_QCed_samples_cat_variables_both_sexes.3.tsv',
                                      missing='',impute=True)

2019-01-14 14:27:53 Hail: INFO: Reading table to impute column types
2019-01-14 14:27:59 Hail: INFO: Finished type imputation
  Loading column 'all_sexes$userId' as type 'int32' (imputed)
  Loading column '22642' as type 'int32' (imputed)
  Loading column '22645' as type 'int32' (imputed)
  Loading column '24009' as type 'int32' (imputed)
  Loading column '24013' as type 'int32' (imputed)
  Loading column '24015' as type 'int32' (imputed)
  Loading column '30150' as type 'int32' (imputed)
  Loading column '30160' as type 'int32' (imputed)
  Loading column '30170' as type 'int32' (imputed)
  Loading column '30230' as type 'int32' (imputed)
  Loading column '40009' as type 'int32' (imputed)
  Loading column '22604_3' as type 'bool' (imputed)
  Loading column '22604_4' as type 'bool' (imputed)
  Loading column '22604_2' as type 'bool' (imputed)
  Loading column '22604_1' as type 'bool' (imputed)
  Loading column '22606_1' as type 'bool' (imputed)
  Loading column '22606_0' as type 'bool' 

IncompleteParseError: Rule 'type' matched in its entirety, but it didn't consume all the text. The non-matching portion of the text begins with 'uct{all_sexes$userId' (line 1, column 4).

In [2]:
# phesant_still_more1 = hl.import_table('gs://ukb31063-mega-gwas/phenotype-files/still-more-phesant/neale_lab_parsed_and_restricted_to_QCed_samples_cat_variables_both_sexes.1.tsv',
#                                       missing='',impute=True,types={'all_sexes$userId':hl.tstr}).rename({'all_sexes$userId':'s'})
# phesant_still_more2 = hl.import_table('gs://ukb31063-mega-gwas/phenotype-files/still-more-phesant/neale_lab_parsed_and_restricted_to_QCed_samples_cat_variables_both_sexes.2.tsv',
#                                       missing='',impute=True,types={'all_sexes$userId':hl.tstr})

Initializing Spark and Hail with default parameters...
Running on Apache Spark version 2.2.1
SparkUI available at http://10.128.0.124:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.7-1329a6224fc7
LOGGING: writing to /home/hail/hail-20190113-2208-0.2.7-1329a6224fc7.log
2019-01-13 22:08:54 Hail: INFO: Reading table to impute column types
2019-01-13 22:09:07 Hail: INFO: Finished type imputation
  Loading column 'all_sexes$userId' as type 'str' (user-specified)
  Loading column '134' as type 'int32' (imputed)
  Loading column '135' as type 'int32' (imputed)
  Loading column '136' as type 'int32' (imputed)
  Loading column '137' as type 'int32' (imputed)
  Loading column '398' as type 'int32' (imputed)
  Loading column '403' as type 'int32' (imputed)
  Loading column '709' as type 'int32' (imputed)
  Loading column '728' as type 'int32' (imputed)
  Loading column '738' as type 'int32' (imputed)
  Loading column '777' as type 'i

IncompleteParseError: Rule 'type' matched in its entirety, but it didn't consume all the text. The non-matching portion of the text begins with 'uct{all_sexes$userId' (line 1, column 4).

In [6]:
std_phesant_ls = [x.strip('"') for x in list(phen_tb_all.row)]
icd_ls = list(icd_phens.row)
finngen_ls = list(finngen.row)
more_phesant_ls = list(phesant_more.row)
all_phens = std_phesant_ls+icd_ls+finngen_ls+more_phesant_ls

In [9]:
i = 0
std_phesant_ct = 0
icd_ct = 0
finngen_ct = 0
more_phesant_ct = 0
for phen in heritable_phens_ls:
    if 'irnt' in phen:
        phen = phen[:len(phen)-5]
    if phen not in all_phens:
            print(phen)
            i += 1
    if phen in std_phesant_ls:
        std_phesant_ct += 1
    elif phen in icd_ls:
        icd_ct += 1
    elif phen in finngen_ls:
        finngen_ct += 1
    elif phen in more_phesant_ls:
        print('      '+phen)
        more_phesant_ct += 1
print('Number of Missing Phens: '+str(i))
print('# in std_phesant: '+str(std_phesant_ct))
print('# in icd10: '+str(icd_ct))
print('# in finngen: '+str(finngen_ct))
print('# in more_phesant: '+str(more_phesant_ct))

1538_0
4968
5556
1468_3
1150_2
20529
22608_2
20116_0
20531
22609_2
1508_2
1418_2
20117_2
22506_114
22610_1
4957
20536_0
670_1
2247_0
4979
1428_3
4990
5001
5012
22617_2314
680_1
      22704
1448_3
4803_0
22606_2
2664_5
22607_0
22607_1
20126_0
Number of Missing Phens: 33
# in std_phesant: 553
# in icd10: 26
# in finngen: 46
# in more_phesant: 1


# Test code

In [7]:
phen_set = 'stillmore1'

heritable_phens_tb = hl.import_table('gs://nbaya/split/ukb_phenos_for_sex_rg.tsv')
#heritable_phens_tb = hl.import_table('/Users/nbaya/Downloads/ukb_phenos_for_sex_rg.tsv')
heritable_phens_ls = [x.phenotype.strip('_irnt') for x in heritable_phens_tb.select('phenotype').collect()] #list of heritable phenotypes for UKBB round 2

if phen_set == 'phesant':
    phen_tb_all = hl.import_table('gs://phenotype_31063/ukb31063.phesant_phenotypes.both_sexes.tsv.bgz',
                                          missing='',impute=True,types={'"userId"': hl.tstr}).rename({ '"userId"': 's'})
elif phen_set == 'finngen':
    phen_tb_all = hl.import_table('gs://ukb31063-mega-gwas/phenotype-files/curated-phenotypes/2018-04-06_ukb-finngen-pheno-for-analysis.tsv',
                                      missing='',impute=True, types={'eid':hl.tstr}).rename({'eid':'s'})
elif phen_set == 'icd10':
    phen_tb_all = hl.import_table('gs://nbaya/sex_linreg/icd10_phenotypes.both_sexes.tsv.bgz',
                                      missing='',impute=True,types={'s': hl.tstr})
elif phen_set == 'more_phesant':
    phen_tb_all = hl.import_table('gs://ukb31063-mega-gwas/phenotype-files/still-more-phesant/cts_irnt.both_sexes.tsv',
                                      missing='',impute=True, types = {'userId':hl.tstr}).rename({ 'userId': 's'})
elif phen_set == 'stillmore1':
    phen_tb_all = hl.import_table('gs://ukb31063-mega-gwas/phenotype-files/still-more-phesant/neale_lab_parsed_and_restricted_to_QCed_samples_cat_variables_both_sexes.1.tsv',
                                  impute=True, missing="",types={'all_sexes$userId': hl.tstr}).rename({'all_sexes$userId':'s'})
elif phen_set == 'stillmore2':
    phen_tb_all = hl.import_table('gs://ukb31063-mega-gwas/phenotype-files/still-more-phesant/neale_lab_parsed_and_restricted_to_QCed_samples_cat_variables_both_sexes.2.tsv',
                                  impute=True, missing="",types={'all_sexes$userId': hl.tstr}).rename({'all_sexes$userId':'s'})
elif phen_set == 'stillmore3':
    phen_tb_all = hl.import_table('gs://ukb31063-mega-gwas/phenotype-files/still-more-phesant/neale_lab_parsed_and_restricted_to_QCed_samples_cat_variables_both_sexes.3.tsv',
                                  impute=True, missing="",types={'all_sexes$userId': hl.tstr}).rename({'all_sexes$userId':'s'})
    
withdrawn = hl.import_table('gs://nbaya/w31063_20181016.csv',missing='',no_header=True)
withdrawn_set = set(withdrawn.f0.take(withdrawn.count()))
phen_tb_all = phen_tb_all.filter(hl.literal(withdrawn_set).contains(phen_tb_all['s']),keep=False) 
phen_tb_all = phen_tb_all.key_by('s')

phen_tb_all_cols = [x.strip('"') for x in list(phen_tb_all.row)]

completed_ls = os.popen('gsutil ls gs://nbaya/rg_sex/allphens/*').read().split('\n')
completed_ls = completed_ls[1:len(completed_ls)-1]
completed_phens = [x.split('_meta')[0].split('/')[5] for x in completed_ls if 'meta_B' in x]

phens_ls = list(set(heritable_phens_ls).intersection(set(phen_tb_all_cols)).difference(set(completed_phens)))

if len(phens_ls) == 0:
    print('\n####################')
    print(phen_set+' is complete')
    print('\n####################')

idx_ls = range(1-1,len(phens_ls), 1) #hard-parallelized

variants = hl.read_matrix_table('gs://nbaya/split/ukb31063.hm3_variants.gwas_samples_repart.mt') #hm3 variants matrix table

print('\n####################')
print('phen set: '+phen_set)
print('parsplit: '+str(1))
print('paridx: '+str(1))
print('n chunks: '+str(n_chunks))
print('batch: '+str(batch))
print('running '+str(len(phens_ls))+' phenotypes: '+'\t'.join(phens_ls[1-1::1]))
print('####################')

2019-01-15 15:29:05 Hail: INFO: Reading table with no type imputation
  Loading column 'phenotype' as type 'str' (type not specified)
  Loading column 'description' as type 'str' (type not specified)
  Loading column 'h2_liability' as type 'str' (type not specified)
  Loading column 'h2_liability_se' as type 'str' (type not specified)
  Loading column 'h2_observed' as type 'str' (type not specified)
  Loading column 'h2_observed_se' as type 'str' (type not specified)
  Loading column 'h2_p' as type 'str' (type not specified)
  Loading column 'h2_sig' as type 'str' (type not specified)
  Loading column 'conf_simple' as type 'str' (type not specified)
  Loading column 'notes' as type 'str' (type not specified)
  Loading column 'intercept' as type 'str' (type not specified)
  Loading column 'intercept_se' as type 'str' (type not specified)
  Loading column 'intercept_p' as type 'str' (type not specified)
  Loading column 'lambdaGC' as type 'str' (type not specified)
  Loading column 'mean

2019-01-15 15:29:17 Hail: INFO: Reading table with no type imputation
  Loading column 'f0' as type 'str' (type not specified)




####################
phen set: stillmore1
parsplit: 1
paridx: 1
n chunks: 2
batch: 1
running 19 phenotypes: 2247_0	1538_0	4957	5001	1468_3	1508_2	4990	1428_3	4979	5012	2664_5	1150_2	1448_3	670_1	4803_0	1418_2	4968	5556	680_1
####################


In [19]:
phen_i = 0
phen = phens_ls[phen_i]
print('####################')
print('Starting phen '+phen+' ('+str(phen_i+1)+' of '+str(len(phens_ls))+' '+phen_set+' phens)')
print('####################')
phen_starttime = datetime.datetime.now()

"""
╔═══════════════╗
║ Preprocessing ║
╚═══════════════╝
"""

if phen_set == 'phesant':
    phen_tb_all1 = phen_tb_all.rename({'"'+phen+'"': 'phen'})
else:
    phen_tb_all1 = phen_tb_all.rename({phen: 'phen'})
phen_tb = phen_tb_all1.select(phen_tb_all1['phen'])

####################
Starting phen 2247_0 (1 of 19 stillmore1 phens)
####################


In [20]:
mt1 = variants.annotate_cols(phen_str = hl.str(phen_tb[variants.s]['phen']).replace('\"',''))
mt1 = mt1.filter_cols(mt1.phen_str == '',keep=False)

In [21]:
if phen_tb.phen.dtype == hl.dtype('bool'):
    mt1 = mt1.annotate_cols(phen = hl.bool(mt1.phen_str)).drop('phen_str')
else:
    mt1 = mt1.annotate_cols(phen = hl.float64(mt1.phen_str)).drop('phen_str')


In [22]:
n_samples = mt1.count_cols()
print('\n>>> phen '+phen+': N samples = '+str(n_samples)+' <<<') #expect n samples to match n_non_missing from phenotypes.both_sexes.tsv

2019-01-15 15:40:03 Hail: INFO: Coerced sorted dataset
2019-01-15 15:40:24 Hail: INFO: Coerced sorted dataset



>>> phen 2247_0: N samples = 346588 <<<


In [23]:
mt2 = mt1.add_col_index()
group_size = int(n_samples/n_chunks)+1     #the ideal number of samples in each group
group_ids = np.ndarray.tolist(np.ndarray.flatten(np.asarray([range(n_chunks)]*group_size))) #list of group ids to be paired to each sample (Note: length of group_ids > # of cols in mt, but it doesn't affect the result)
group_ids = group_ids[0:n_samples]
randstate = np.random.RandomState(int(batch)) #seed with batch number
randstate.shuffle(group_ids)
mt3 = mt2.annotate_cols(group_id = hl.literal(group_ids)[hl.int32(mt2.col_idx)]) #assign group ids

mt = mt3.rename({'dosage': 'x', 'phen': 'y'})

2019-01-15 15:40:37 Hail: INFO: Coerced sorted dataset


In [24]:
mt_A = mt.filter_cols(mt.group_id == 0)
mt_B = mt.filter_cols(mt.group_id == 1)

cov_list_A = [ mt_A['isFemale'], mt_A['age'], mt_A['age_squared'], mt_A['age_isFemale'],
        mt_A['age_squared_isFemale'] ]+ [mt_A['PC{:}'.format(i)] for i in range(1, 21)] 

cov_list_B = [ mt_B['isFemale'], mt_B['age'], mt_B['age_squared'], mt_B['age_isFemale'],
        mt_B['age_squared_isFemale'] ]+ [mt_B['PC{:}'.format(i)] for i in range(1, 21)] 

ht_A = hl.linear_regression_rows(
        y=mt_A.y,
        x=mt_A.x,
        covariates=[1]+cov_list_A,
        pass_through = ['rsid'])

ht_B = hl.linear_regression_rows(
        y=mt_B.y,
        x=mt_B.x,
        covariates=[1]+cov_list_B,
        pass_through = ['rsid'])

ht_A = ht_A.rename({'rsid':'SNP'}).key_by('SNP')
ht_B = ht_B.rename({'rsid':'SNP'}).key_by('SNP')

ht_A = ht_A.select(Z = ht_A.beta/ht_A.standard_error)
ht_B = ht_B.select(Z = ht_B.beta/ht_B.standard_error)

sumstats_template = hl.import_table('gs://nbaya/rg_sex/50_snps_alleles_N.tsv.gz',types={'N': hl.tint64})
sumstats_template = sumstats_template.key_by('SNP')
sumstats_template = sumstats_template.annotate(N = int(n_samples/2))

metaA = sumstats_template.annotate(Z = ht_A[sumstats_template.SNP]['Z'])
metaB = sumstats_template.annotate(Z = ht_B[sumstats_template.SNP]['Z'])

metaA_path = output_bucket+phen+'_meta_A_nchunks'+str(n_chunks)+'_batch'+str(batch)+'_split.tsv.bgz' 
metaB_path = output_bucket+phen+'_meta_B_nchunks'+str(n_chunks)+'_batch'+str(batch)+'_split.tsv.bgz' 
metaA.export(metaA_path)
metaB.export(metaB_path)

print('\n####################')
print('Files written to:')
print(metaA_path+'\t'+metaB_path)
print('\n####################')

2019-01-15 15:44:22 Hail: INFO: linear_regression_rows: running on 173294 samples for 1 response variable y,
    with input variable x, and 26 additional covariates...


KeyboardInterrupt: 

# Sex strat random split

In [24]:
n_chunks = 2 #number of partitions of the dataset
batch = 1 #This is used for the seed to replicate the random partitioning. default: 1
phen_set = 'phesant'
phen = '50'

output_bucket = 'gs://nbaya/rg_sex/sex_strat/'

In [27]:
heritable_phens_tb = hl.import_table('gs://nbaya/split/ukb_phenos_for_sex_rg.tsv')
#heritable_phens_tb = hl.import_table('/Users/nbaya/Downloads/ukb_phenos_for_sex_rg.tsv')
heritable_phens_ls = [x.phenotype.strip('_irnt') for x in heritable_phens_tb.select('phenotype').collect()] #list of heritable phenotypes for UKBB round 2

if phen_set == 'phesant':
    phen_tb_all = hl.import_table('gs://phenotype_31063/ukb31063.phesant_phenotypes.both_sexes.tsv.bgz',
                                          missing='',impute=True,types={'"userId"': hl.tstr}).rename({ '"userId"': 's'})

withdrawn = hl.import_table('gs://nbaya/w31063_20181016.csv',missing='',no_header=True)
withdrawn_set = set(withdrawn.f0.take(withdrawn.count()))
phen_tb_all = phen_tb_all.filter(hl.literal(withdrawn_set).contains(phen_tb_all['s']),keep=False) 
phen_tb_all = phen_tb_all.key_by('s')    
    
variants = hl.read_matrix_table('gs://nbaya/split/ukb31063.hm3_variants.gwas_samples_repart.mt') #hm3 variants matrix table

# if phen_set == 'phesant':
#     phen_tb_all1 = phen_tb_all.rename({'"'+phen+'"': 'phen'})
# else:
#     phen_tb_all1 = phen_tb_all.rename({phen: 'phen'})
# phen_tb = phen_tb_all1.select(phen_tb_all1['phen'])

# mt1 = variants.annotate_cols(phen_str = hl.str(phen_tb[variants.s]['phen']).replace('\"',''))
# mt1 = mt1.filter_cols(mt1.phen_str == '',keep=False)

# if phen_tb.phen.dtype == hl.dtype('bool'):
#     mt1 = mt1.annotate_cols(phen = hl.bool(mt1.phen_str)).drop('phen_str')
# else:
#     mt1 = mt1.annotate_cols(phen = hl.float64(mt1.phen_str)).drop('phen_str')

2019-01-17 13:39:22 Hail: INFO: Reading table with no type imputation
  Loading column 'phenotype' as type 'str' (type not specified)
  Loading column 'description' as type 'str' (type not specified)
  Loading column 'h2_liability' as type 'str' (type not specified)
  Loading column 'h2_liability_se' as type 'str' (type not specified)
  Loading column 'h2_observed' as type 'str' (type not specified)
  Loading column 'h2_observed_se' as type 'str' (type not specified)
  Loading column 'h2_p' as type 'str' (type not specified)
  Loading column 'h2_sig' as type 'str' (type not specified)
  Loading column 'conf_simple' as type 'str' (type not specified)
  Loading column 'notes' as type 'str' (type not specified)
  Loading column 'intercept' as type 'str' (type not specified)
  Loading column 'intercept_se' as type 'str' (type not specified)
  Loading column 'intercept_p' as type 'str' (type not specified)
  Loading column 'lambdaGC' as type 'str' (type not specified)
  Loading column 'mean

  Loading column '"100005"' as type 'float64' (imputed)
  Loading column '"100006"' as type 'float64' (imputed)
  Loading column '"100007"' as type 'float64' (imputed)
  Loading column '"100008"' as type 'float64' (imputed)
  Loading column '"100009"' as type 'float64' (imputed)
  Loading column '"100010"' as type 'str' (imputed)
  Loading column '"100011"' as type 'float64' (imputed)
  Loading column '"100012"' as type 'float64' (imputed)
  Loading column '"100013"' as type 'float64' (imputed)
  Loading column '"100014"' as type 'float64' (imputed)
  Loading column '"100015"' as type 'float64' (imputed)
  Loading column '"100016"' as type 'float64' (imputed)
  Loading column '"100017"' as type 'float64' (imputed)
  Loading column '"100018"' as type 'float64' (imputed)
  Loading column '"100019"' as type 'float64' (imputed)
  Loading column '"100021"' as type 'float64' (imputed)
  Loading column '"100022"' as type 'str' (imputed)
  Loading column '"100023"' as type 'float64' (imputed)


In [28]:
if phen_set == 'phesant':
    phen_tb_all1 = phen_tb_all.rename({'"'+phen+'"': 'phen'})
else:
    phen_tb_all1 = phen_tb_all.rename({phen: 'phen'})
phen_tb = phen_tb_all1.select(phen_tb_all1['phen'])

mt1 = variants.annotate_cols(phen_str = hl.str(phen_tb[variants.s]['phen']).replace('\"',''))
mt1 = mt1.filter_cols(mt1.phen_str == '',keep=False)

if phen_tb.phen.dtype == hl.dtype('bool'):
    mt1 = mt1.annotate_cols(phen = hl.bool(mt1.phen_str)).drop('phen_str')
else:
    mt1 = mt1.annotate_cols(phen = hl.float64(mt1.phen_str)).drop('phen_str')

for sex in ['female','male']:
    mt2 = mt1.filter_cols(mt1.isFemale == (sex == 'female'))

    mt3 = mt2.add_col_index()
    mt3 = mt3.rename({'dosage': 'x', 'phen': 'y'})

    n_samples = mt3.count_cols()
    print('\n>>> phen '+phen+': N samples = '+str(n_samples)+' <<<') 

2019-01-17 13:40:31 Hail: INFO: Coerced sorted dataset



>>> phen 50: N samples = 193755 <<<


2019-01-17 13:41:53 Hail: INFO: Coerced sorted dataset



>>> phen 50: N samples = 166583 <<<


In [26]:
sex = 'female'
mt2 = mt1.filter_cols(mt1.isFemale == (sex == 'female'))
        
mt3 = mt2.add_col_index()
mt3 = mt3.rename({'dosage': 'x', 'phen': 'y'})

n_samples = mt3.count_cols()
print('\n>>> phen '+phen+': N samples = '+str(n_samples)+' <<<') 

group_size = int(n_samples/n_chunks)+1     #the ideal number of samples in each group
group_ids = np.ndarray.tolist(np.ndarray.flatten(np.asarray([range(n_chunks)]*group_size))) #list of group ids to be paired to each sample (Note: length of group_ids > # of cols in mt, but it doesn't affect the result)
group_ids = group_ids[0:n_samples]
randstate = np.random.RandomState(int(batch)) #seed with batch number
randstate.shuffle(group_ids)
mt = mt3.annotate_cols(group_id = hl.literal(group_ids)[hl.int32(mt3.col_idx)]) #assign group ids

2019-01-17 13:37:23 Hail: INFO: Coerced sorted dataset



>>> phen 50: N samples = 193755 <<<


In [21]:
group_size = int(n_samples/n_chunks)+1     #the ideal number of samples in each group
group_ids = np.ndarray.tolist(np.ndarray.flatten(np.asarray([range(n_chunks)]*group_size))) #list of group ids to be paired to each sample (Note: length of group_ids > # of cols in mt, but it doesn't affect the result)
group_ids = group_ids[0:n_samples]
randstate = np.random.RandomState(int(batch)) #seed with batch number
randstate.shuffle(group_ids)
mt = mt3.annotate_cols(group_id = hl.literal(group_ids)[hl.int32(mt3.col_idx)]) #assign group ids

In [22]:
mt_A = mt.filter_cols(mt.group_id == 0)
mt_B = mt.filter_cols(mt.group_id == 1)

In [23]:
mt_A.count_cols()

2019-01-17 13:13:29 Hail: INFO: Coerced sorted dataset


96099

mt_A